In [21]:
import os
import pandas as pd
import sys
import seaborn as sns
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy.stats as stats
from creds import getCensusAPI

path = os.path.abspath('')


In [22]:
#Load raw tract-wise data
fcc_path = path+'\\static-data\\cleaned_fcc_tracts.csv'

fcc_data = pd.read_csv(fcc_path)

fcc_data.drop(['Unnamed: 0'], axis=1, inplace=True)

fcc_data['state'] = fcc_data['state'].astype(str).str.zfill(2)
fcc_data['tract'] = fcc_data['tract'].astype(str).str.zfill(6)
fcc_data['county'] = fcc_data['county'].astype(str).str.zfill(3)

fcc_data.describe()

,Max Advertised Downstream Speed (mbps)
count,72987.000000
mean,838.148725
std,725.073403
min,2.000000
25%,940.000000
50%,1000.000000
75%,1000.000000
max,10000.000000


In [23]:
#Current threshold = 50mbps, further narrow down if needed, add labels to blocks
threshold = 50

fcc_data['Access'] = np.nan
fcc_data['Access'][fcc_data['Max Advertised Downstream Speed (mbps)'] >= threshold] = 1
fcc_data['Access'][fcc_data['Max Advertised Downstream Speed (mbps)'] < threshold] = 0

C:\Users\lkyoo\AppData\Local\Temp\ipykernel_7316\1412316767.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fcc_data['Access'][fcc_data['Max Advertised Downstream Speed (mbps)'] >= threshold] = 1
C:\Users\lkyoo\AppData\Local\Temp\ipykernel_7316\1412316767.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fcc_data['Access'][fcc_data['Max Advertised Downstream Speed (mbps)'] < threshold] = 0


In [24]:
print('{}% of census tracts lack access to sufficient internet speeds'.format(round((fcc_data['Access'].value_counts()[0]/len(fcc_data['Access']))*100,2)))

9.71% of census tracts lack access to sufficient internet speeds


https://api.census.gov/data/2020/acs/acs5/variables.html

In [25]:
cols = {
    #Population
    'B01003_001E':'Total',
    #Sex
    'B01001_002E':'Total Male',
    'B01001_026E':'Total Female',
    #Age
    'B01002_001E': 'Median Age',
    #Race 
    'C02003_003E': 'White',
    'C02003_004E': 'Black or African American',
    'C02003_005E': 'American Indian and Alaska Native',
    'C02003_006E': 'Asian',
    'C02003_007E': 'Native Hawaiian and Other Pacific Islander',
    'C02003_008E': 'Some other race',
    'C02003_009E': 'Two or more races',
    'C02003_010E': 'Two or more including some other race',
    'C02003_011E': 'Two or more excluding some other race, and three or more',
    'B03001_003E': 'Hispanic or Latino',
    #Citizenship
    'B05001_006E': 'Not a Citizen',
    #Birth
    'B05012_002E': 'Native',
    'B05012_003E': 'Foreign Born',
    #English and language proficiency
    'B06007_002E': 'Speak only English',
    'B06007_003E': 'Speak Spanish',
    'B06007_004E': 'Speak Spanish, English very well',
    'B06007_005E': 'Speak Spanish, English less than very well',
    'B06007_006E': 'Speak other languages',
    'B06007_007E': 'Speak other language, English very well',
    'B06007_008E': 'Speak other language, English less than very well',
    #Individual income
    'B06010_002E': 'No individual income (12mo)',
    'B06010_003E': 'With individual income (12mo)',
    'B06010_004E': 'Less than 10000 individual income (12mo)',
    'B06010_005E': '10000 - 14999 individual income (12mo)',
    'B06010_006E': '15000 - 24999 individual income (12mo)',
    'B06010_007E': '25000 - 34999 individual income (12mo)',
    'B06010_008E': '35000 - 49999 individual income (12mo)',
    'B06010_009E': '50000 - 64999 individual income (12mo)',
    'B06010_010E': '65000 - 74999 individual income (12mo)',
    'B06010_011E': '75000 or more individual income (12mo)',
    'B06011_001E': 'Median individual income (12mo)',
    #Poverty level comparisons
    'B05010_002E': 'Ratio of income to poverty < 1.00',
    'B05010_010E': 'Ratio of income to poverty 1.00 to 1.99',
    'B05010_018E': 'Ratio of income to poverty > 1.99',
    'B06012_002E': 'Below 100% of poverty level',
    'B06012_003E': '100 to 149% of poverty level',
    'B06012_004E': 'At or above 150% of poverty level',
    #Geographical mobility
    'B07001_001E': 'Moved in the last year',
    'B07001_017E': 'Same house 1 year ago',
    'B07001_033E': 'Moved within same county',
    'B07001_049E': 'Moved from different county within same state',
    'B07001_065E': 'Moved from different state',
    'B07001_081E': 'Moved from abroad',
}

In [26]:
cols2 = {
    #Education
    'B07009_002E': 'Less than High School',
    'B06009_003E': 'High School Grad',
    'B06009_004E': 'Some college or associate degree',
    'B06009_005E': 'Bachelor',
    'B06009_006E': 'Graduate or professional degree',
    #Educational Attainment for 25 and over
    'B15003_002E': 'No schooling completed (25 and over)',
    'B15003_003E': 'Nursery (25 and over)',
    'B15003_004E': 'Kindergarten (25 and over)',
    'B15003_005E': '1st grade (25 and over)',
    'B15003_006E': '2nd grade (25 and over)',
    'B15003_007E': '3rd grade (25 and over)',
    'B15003_008E': '4th grade (25 and over)',
    'B15003_009E': '5th grade (25 and over)',
    'B15003_010E': '6th grade (25 and over)',
    'B15003_011E': '7th grade (25 and over)',
    'B15003_012E': '8th grade (25 and over)',
    'B15003_013E': '9th grade (25 and over)',
    'B15003_014E': '10th grade (25 and over)',
    'B15003_015E': '11th grade (25 and over)',
    'B15003_016E': '12th grade no diploma (25 and over)',
    'B15003_017E': 'HS Diploma (25 and over)',
    'B15003_018E': 'GED or alternative (25 and over)',
    'B15003_019E': 'Some college, less than 1 year (25 and over)',
    'B15003_020E': 'Some college, 1 or more years no degree (25 and over)',
    'B15003_021E': 'Associate degree',
    'B15003_022E': 'Bachelor degree',
    'B15003_023E': 'Masters degree',
    'B15003_024E': 'Professional school degree',
    'B15003_025E': 'Doctorate degree',
    #Labor
    'B23025_002E': 'In labor force',
    'B23025_007E': 'Not in labor force',
    'B23020_001E': 'Mean usual hours worked',
    'B08122_025E': 'Worked from home',
    #Occupation
    'B08124_002E': 'Management, business, science, and arts',
    'B08124_003E': 'Service occupations',
    'B08124_004E': 'Sales and office occupations',
    'B08124_005E': 'Natural resources, construction, and maintenance occupations',
    'B08124_006E': 'Production, transportation, and material moving occupations',
    'B08124_007E': 'Military specific occupations',
}

In [27]:
cols3 = {
    #Industry
    'B08126_002E': 'Agriculture, forestry, fishing, hunting, mining',
    'B08126_003E': 'Construction',
    'B08126_004E': 'Manufacturing',
    'B08126_005E': 'Wholesale trade',
    'B08126_006E': 'Retail trade',
    'B08126_007E': 'Transportation and warehousing, and utilities',
    'B08126_008E': 'Information',
    'B08126_009E': 'Finance and insurance, and real estate and rental and leasing',
    'B08126_010E': 'Professional, scientific, and management, and administrative and waste management services',
    'B08126_011E': 'Educational services, and health care and social assistance',
    'B08126_012E': 'Arts, entertainment, and recreation, and accommodation and food services',
    'B08126_013E': 'Other services (except public administration)',
    'B08126_014E': 'Public administration',
    'B08126_015E': 'Armed forces',
    #Inequality
    'B19083_001E': 'Gini Index',
    'B19081_006E': 'Top 5 percent mean income',
    'B19082_006E': 'Top 5 percent income share',
    #Other income streams
    'B19054_002E': 'With interest, rent, dividend',
    'B19055_002E': 'With social security income',
    'B19056_002E': 'With supplement security income (SSI)',
    'B19057_002E': 'With public assistance income',
    'B19058_002E': 'With public assistance income or food stamps',
    #Family income
    'B19101_002E': 'Family income <10000 (12mo)',
    'B19101_003E': 'Family income 10000 - 14999 (12mo)',
    'B19101_004E': 'Family income 14000 - 19999 (12mo)',
    'B19101_005E': 'Family income 20000 - 24999 (12mo)',
    'B19101_006E': 'Family income 25000 - 29999 (12mo)',
    'B19101_007E': 'Family income 30000 - 34999 (12mo)',
    'B19101_008E': 'Family income 35000 - 39999 (12mo)',
    'B19101_009E': 'Family income 40000 - 44999 (12mo)',
    'B19101_010E': 'Family income 45000 - 49999 (12mo)',
    'B19101_011E': 'Family income 50000 - 59999 (12mo)',
    'B19101_012E': 'Family income 60000 - 74999 (12mo)',
    'B19101_013E': 'Family income 75000 - 99999 (12mo)',
    'B19101_014E': 'Family income 100000 - 124999 (12mo)',
    'B19101_015E': 'Family income 125000 - 149999 (12mo)',
    'B19101_016E': 'Family income 150000 - 199999 (12mo)',
    'B19101_017E': 'Family income >200000 (12mo)',
}

In [28]:
cols4 = {
    #Median earning by occupation
    'B24011_004E': 'Median earnings for management',
    'B24011_005E': 'Median earnings for business and financial ops',
    'B24011_006E': 'Median earnings for computer, engineering and science',
    'B24011_007E': 'Median earnings for computer and mathematical',
    'B24011_008E': 'Median earnings for architecture and engineering',
    'B24011_009E': 'Median earnings for life, physical, and social science',
    'B24011_011E': 'Median earnings for community and social service',
    'B24011_012E': 'Median earnings for legal',
    'B24011_013E': 'Median earnings for educational instruction and library',
    'B24011_014E': 'Median earnings for arts, design, entertainment, sports, and media',
    'B24011_016E': 'Median earnings for health diagnosing and treating practitioners and other technical occupations',
    'B24011_017E': 'Median earnings for health technologists and technicians',
    'B24011_019E': 'Median earnings for healthcare support occupations',
    'B24011_022E': 'Median earnings for law enforcement workers',
    'B24011_022E': 'Median earnings for food preparation and serving',
    'B24011_024E': 'Median earnings for building and grounds cleaning and maintenance',
    'B24011_025E': 'Median earnings for personal care and service',
    'B24011_027E': 'Median earnings for sales and related occupations',
    'B24011_028E': 'Median earnings for office and administrative support',
    'B24011_030E': 'Median earnings for farming, fishing, and forestry',
    'B24011_031E': 'Median earnings for construction and extraction',
    'B24011_032E': 'Median earnings for installation, maintenance, and repair',
    'B24011_034E': 'Median earnings for production',
    'B24011_035E': 'Median earnings for transportation',
    'B24011_036E': 'Median earnings for material moving',
    #Property
    'B25077_001E': 'Median property value',
    'B25111_001E': 'Median gross rent',
}

In [29]:
#Fuse with ACS data

#Set of columns
col_set = [cols, cols2, cols3, cols4]

#Set API Key
api_key = getCensusAPI()

#State set
states = fcc_data['state'].unique()

acs_data = pd.DataFrame({'A' : []})


for vals in col_set:
    temp_acs_data = pd.DataFrame({'A' : []})
    #Get bulk block group level data
    for state in states: 
        stateText = str(state).zfill(2)

        url = 'https://api.census.gov/data/2020/acs/acs5?get={}&for=tract:*&in=state:{}&key={}'.format(','.join(vals.keys()),stateText, api_key)

        temp_data = pd.read_json(url)

        if temp_acs_data.empty: 
            temp_acs_data = temp_data
        else: 
            temp_acs_data = pd.concat([temp_acs_data,temp_data])

    temp_acs_data.columns = temp_acs_data.iloc[0]

    temp_acs_data.drop([0], inplace=True)
    
    temp_acs_data.rename(columns=vals, inplace=True)

    if acs_data.empty: 
        acs_data = temp_acs_data
    else: 
        acs_data = pd.merge(acs_data,temp_acs_data,on=['state','county','tract'], how='left')

#Change numerical columns to floats
exclude_cols = ['state', 'county', 'tract']
columns = acs_data.columns

for col in columns:
    if not col in exclude_cols:
        acs_data[col] = acs_data[col].astype(float)

acs_data.replace(-666666666.0, 0,inplace=True)

#Change identifier to strings, and standardize with fcc data
acs_data['state'] = acs_data['state'].astype(str).str.zfill(2)
acs_data['tract'] = acs_data['tract'].astype(str).str.zfill(6)
acs_data['county'] = acs_data['county'].astype(str).str.zfill(3)

#Create derived columns
acs_data['Non-white'] = acs_data['Total'] - acs_data['White']
acs_data['U.S. Citizen'] = acs_data['Total'] - acs_data['Not a Citizen']

acs_data[:10]

,Total,Total Male,Total Female,Median Age,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,...,"Median earnings for farming, fishing, and forestry",Median earnings for construction and extraction,"Median earnings for installation, maintenance, and repair",Median earnings for production,Median earnings for transportation,Median earnings for material moving,Median property value,Median gross rent,Non-white,U.S. Citizen
0,4385.0,1788.0,2597.0,48.0,145.0,4170.0,0.0,0.0,0.0,70.0,...,0.0,31006.0,0.0,30720.0,27232.0,18920.0,85600.0,968.0,4240.0,4362.0
1,4285.0,2039.0,2246.0,34.8,185.0,3961.0,0.0,26.0,0.0,21.0,...,0.0,25556.0,0.0,17500.0,46797.0,13789.0,65200.0,792.0,4100.0,4253.0
2,5474.0,2152.0,3322.0,29.7,96.0,5300.0,0.0,0.0,0.0,0.0,...,0.0,20362.0,0.0,27951.0,0.0,21889.0,68700.0,868.0,5378.0,5432.0
3,3514.0,1911.0,1603.0,49.1,146.0,3110.0,54.0,6.0,0.0,75.0,...,0.0,23401.0,0.0,50183.0,0.0,28500.0,68900.0,913.0,3368.0,3460.0
4,1265.0,509.0,756.0,38.4,113.0,998.0,0.0,10.0,0.0,99.0,...,0.0,27171.0,0.0,23558.0,65500.0,16250.0,57400.0,668.0,1152.0,1156.0
5,2533.0,1124.0,1409.0,53.3,248.0,2104.0,11.0,0.0,0.0,13.0,...,0.0,31994.0,0.0,48646.0,0.0,30333.0,73000.0,617.0,2285.0,2452.0
6,1625.0,832.0,793.0,44.7,229.0,1386.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,28882.0,0.0,0.0,69200.0,566.0,1396.0,1569.0
7,2650.0,829.0,1821.0,19.7,1531.0,721.0,45.0,90.0,0.0,77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763.0,1119.0,2521.0
8,3109.0,1370.0,1739.0,28.0,1268.0,1141.0,0.0,465.0,0.0,158.0,...,0.0,0.0,0.0,14643.0,0.0,17594.0,253600.0,1160.0,1841.0,2746.0
9,3802.0,1825.0,1977.0,32.8,3046.0,493.0,0.0,241.0,0.0,0.0,...,0.0,0.0,0.0,0.0,175000.0,0.0,228700.0,937.0,756.0,3568.0


In [30]:
np.shape(acs_data)

(84414, 155)

In [31]:
#Fuse data
pd_fused = pd.merge(fcc_data,acs_data,on=['state','county','tract'], how='left')

pd_fused

,state,county,tract,Max Advertised Downstream Speed (mbps),Access,Total,Total Male,Total Female,Median Age,White,...,"Median earnings for farming, fishing, and forestry",Median earnings for construction and extraction,"Median earnings for installation, maintenance, and repair",Median earnings for production,Median earnings for transportation,Median earnings for material moving,Median property value,Median gross rent,Non-white,U.S. Citizen
0,01,001,020100,940.0,1.0,1941.0,978.0,963.0,38.0,1537.0,...,0.0,52917.0,63750.0,57917.0,34167.0,0.0,151200.0,820.0,404.0,1937.0
1,01,001,020200,1000.0,1.0,1757.0,1010.0,747.0,35.6,647.0,...,0.0,0.0,0.0,45833.0,22386.0,0.0,85700.0,846.0,1110.0,1757.0
2,01,001,020300,1000.0,1.0,3694.0,1839.0,1855.0,35.5,2363.0,...,0.0,29055.0,15179.0,26722.0,19358.0,34231.0,115000.0,937.0,1331.0,3539.0
3,01,001,020400,1000.0,1.0,3539.0,1794.0,1745.0,47.7,3097.0,...,0.0,34420.0,60125.0,32100.0,42991.0,30074.0,163100.0,917.0,442.0,3465.0
4,01,001,020500,1000.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72982,09,015,906100,940.0,1.0,5080.0,2592.0,2488.0,45.8,4775.0,...,0.0,24671.0,63889.0,34728.0,12361.0,42581.0,233900.0,973.0,305.0,5008.0
72983,09,015,907100,1000.0,1.0,4664.0,2361.0,2303.0,44.8,4196.0,...,0.0,73047.0,58553.0,47115.0,38656.0,32411.0,189600.0,879.0,468.0,4556.0
72984,09,015,907200,1000.0,1.0,5095.0,2544.0,2551.0,35.8,4818.0,...,0.0,42227.0,53864.0,48897.0,0.0,39837.0,188400.0,961.0,277.0,4968.0
72985,09,015,907300,1000.0,1.0,5370.0,2593.0,2777.0,43.8,4794.0,...,0.0,62955.0,39135.0,38892.0,51406.0,11761.0,195400.0,906.0,576.0,5274.0


In [32]:
non_na = pd_fused.fillna(0)

#Remove tracts with no people
non_na['Total'] = non_na['Total'].astype(int)
non_na = non_na[non_na['Total'] > 0]

#Drop identifiers
non_na.drop(['Max Advertised Downstream Speed (mbps)','state','county','tract'], axis=1,inplace=True)

print(non_na.Total.sum())
print(np.shape(non_na))
print('{}% of tracked census tracts lack access to sufficient internet speeds (based on median in block group)'.format(round((non_na['Access'].value_counts()[0]/len(non_na['Access']))*100,2)))
print('{} people lack access to sufficient internet speeds (based on median in block group)'.format(non_na[non_na['Access'] == 0].Total.sum()))

244060186
(60981, 153)
9.2% of tracked census tracts lack access to sufficient internet speeds (based on median in block group)
18943087 people lack access to sufficient internet speeds (based on median in block group)


In [33]:
#Correlation Matrix
from sklearn import preprocessing

#scaler = preprocessing.MinMaxScaler()

#scaled = pd.DataFrame(scaler.fit_transform(non_na))

#scaled.columns = non_na.columns

correlation = non_na.corr()

access_corr = correlation['Access'].sort_values()

access_corr

Agriculture, forestry, fishing, hunting, mining                                              -0.223690
Median earnings for farming, fishing, and forestry                                           -0.218352
American Indian and Alaska Native                                                            -0.114508
Median Age                                                                                   -0.114077
Median earnings for installation, maintenance, and repair                                    -0.076435
                                                                                                ...   
Bachelor                                                                                      0.170167
Bachelor degree                                                                               0.170167
Median gross rent                                                                             0.183555
Professional, scientific, and management, and administrative and waste ma

In [34]:
access_set = non_na[non_na['Access'] == 1]

noaccess_set = non_na[non_na['Access'] == 0]

In [35]:
#OLS Regression Implementation
#import statsmodels.api as sm

#Y = scaled.Access
#X = scaled.loc[:, scaled.columns != 'Access']

#model = sm.OLS(Y,X)

#results = model.fit()

#results.summary()

T Test on mean
https://stackoverflow.com/questions/43292885/how-to-perform-a-t-test-on-data-stored-in-a-pandas-dataframe

Median Test


https://www.kaggle.com/code/omkardhudi/hypothesis-testing-in-python/notebook


In [54]:
#Split out blocks without access and analyze
access = non_na[non_na['Access'] == 1]

no_access = non_na[non_na['Access'] == 0]

from scipy.stats import ttest_ind
from scipy.stats import kruskal

median_comparison = pd.concat([pd.DataFrame(access.median()), pd.DataFrame(no_access.median())], axis=1, ignore_index=True)
mean_comparison = pd.concat([pd.DataFrame(access.mean()), pd.DataFrame(no_access.mean())], axis=1, ignore_index=True)

tt = ttest_ind(access, no_access)
mt = kruskal(access, no_access)

mean_comparison['t statistic'] = tt[0]
mean_comparison['p value'] = np.asarray(np.round(np.array(tt[1]),8), dtype = np.float64, order ='C')

median_comparison['h statistic'] = mt[0]
median_comparison['p value'] = np.asarray(np.round(np.array(mt[1]),8), dtype = np.float64, order ='C')

outlier_mean = mean_comparison[mean_comparison['p value'] >= 0.01]
outlier_median = median_comparison[median_comparison['p value'] >= 0.01]

mean_comparison = mean_comparison[mean_comparison['p value'] < 0.01].sort_values('t statistic')
median_comparison = median_comparison[median_comparison['p value'] < 0.01].sort_values('h statistic')


In [37]:
no_access.describe()

,Access,Total,Total Male,Total Female,Median Age,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,...,"Median earnings for farming, fishing, and forestry",Median earnings for construction and extraction,"Median earnings for installation, maintenance, and repair",Median earnings for production,Median earnings for transportation,Median earnings for material moving,Median property value,Median gross rent,Non-white,U.S. Citizen
count,5612.0,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,...,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5612.000000,5.612000e+03,5612.000000,5612.000000,5612.000000
mean,0.0,3375.460976,1717.483428,1657.977548,42.786725,2629.711867,366.258197,97.854954,64.864754,5.544191,...,10368.739843,37520.141126,37530.037776,34208.340877,33948.870456,19577.115645,1.732540e+05,811.536707,745.749109,3277.025303
std,0.0,1523.731380,833.211017,776.728259,8.403319,1394.964316,685.730605,468.854068,233.639418,55.507214,...,18354.617223,23948.515069,27183.730866,22372.048882,26174.995264,18055.041755,1.632029e+05,422.567368,943.415246,1462.872327
min,0.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,3.000000
25%,0.0,2269.000000,1127.000000,1104.000000,37.900000,1644.000000,5.000000,0.000000,0.000000,0.000000,...,0.000000,24583.000000,16827.750000,22619.000000,16537.500000,0.000000,9.060000e+04,611.000000,131.000000,2210.750000
50%,0.0,3217.500000,1610.500000,1578.000000,42.850000,2540.500000,47.000000,4.000000,7.000000,0.000000,...,0.000000,38324.000000,40662.500000,34745.500000,34143.500000,20254.500000,1.304000e+05,732.000000,384.000000,3127.000000
75%,0.0,4291.000000,2170.250000,2137.000000,47.900000,3517.000000,415.000000,29.000000,38.000000,0.000000,...,20355.500000,50867.500000,53750.000000,45000.000000,48690.750000,30769.750000,1.932250e+05,906.000000,992.000000,4185.000000
max,0.0,17213.000000,8800.000000,9694.000000,89.200000,9172.000000,7590.000000,9934.000000,5476.000000,2354.000000,...,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,2.000001e+06,3501.000000,10181.000000,13975.000000


In [38]:
access.describe()

,Access,Total,Total Male,Total Female,Median Age,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,...,"Median earnings for farming, fishing, and forestry",Median earnings for construction and extraction,"Median earnings for installation, maintenance, and repair",Median earnings for production,Median earnings for transportation,Median earnings for material moving,Median property value,Median gross rent,Non-white,U.S. Citizen
count,55369.0,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,...,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,55369.000000,5.536900e+04,55369.000000,55369.000000,55369.000000
mean,1.0,4065.760606,1994.793711,2070.966895,39.627385,2823.077318,529.867326,27.360093,237.480305,6.962921,...,2247.320233,33589.719572,29873.064404,31537.210172,30171.338095,17748.151565,2.943532e+05,1166.274305,1242.683288,3786.945493
std,0.0,1628.823823,828.835126,845.100761,7.906528,1529.747397,848.251711,110.205274,475.139155,42.494228,...,9332.995258,26809.464536,29039.263020,23770.065470,27237.062285,17291.036954,2.707839e+05,560.399950,1180.001764,1519.542083
min,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1.0,2867.000000,1394.000000,1452.000000,34.400000,1728.000000,39.000000,0.000000,10.000000,0.000000,...,0.000000,9609.000000,0.000000,16839.000000,0.000000,0.000000,1.267000e+05,797.000000,360.000000,2674.000000
50%,1.0,3932.000000,1918.000000,2000.000000,39.200000,2736.000000,173.000000,0.000000,68.000000,0.000000,...,0.000000,33712.000000,31503.000000,32011.000000,29712.000000,17330.000000,2.107000e+05,1052.000000,871.000000,3655.000000
75%,1.0,5104.000000,2503.000000,2603.000000,44.500000,3812.000000,623.000000,22.000000,246.000000,0.000000,...,0.000000,49464.000000,50833.000000,43750.000000,45523.000000,28274.000000,3.671000e+05,1428.000000,1770.000000,4743.000000
max,1.0,39373.000000,27204.000000,12169.000000,85.100000,28428.000000,8200.000000,6073.000000,10250.000000,2649.000000,...,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,2.000001e+06,3501.000000,10945.000000,38519.000000
